In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC, SDS
from pyproj import CRS, Proj, Transformer
import geopandas as gpd
from shapely.geometry import Point
import datetime

In [ ]:
def modis_preprocessor(filelist, location_data, storage_path, modis_data_path):
    """
    filelist: list of modis files to be preprocessed. File list must be sorted by date of observation
    location_data: numpy array containing latitude and longitude of location of interest

    returns: 
        furnished: pandas dataframe with following columns - ['date', 'latitude', 'longitude', 'AOD47', 'AOD47_3', 'valid3', 'AOD47_5', 'valid5']
    
    """

    ls = []
    count = 0

    for i in filelist:
        filename = str(i)

        date = datetime.datetime.strptime(filename[11:16], '%y%j').date().strftime("%d-%m-%Y %H:%M")
        hdf_file = SD(os.path.join(modis_data_path, filename), SDC.READ)

        #collecting required datasets
        AOD_47 = hdf_file.select('Optical_Depth_047')
        AOD_55 = hdf_file.select('Optical_Depth_055')
        AOD_QA = hdf_file.select('AOD_QA')
        CWV = hdf_file.select('Column_WV')

        #seperation and parsing of meta data for grid-1 with 1km spatial resolution
        struct_meta_str = hdf_file.attributes()['StructMetadata.0']
        grid1 = struct_meta_str.split("END_GROUP=GRID_1")[0]
        grid1_meta = dict([x.split("=") for x in grid1.split() if "=" in x])

        ###############################################################

        """ Creation of meshgrid"""

        x0, y0 = eval(grid1_meta["UpperLeftPointMtrs"])
        x1, y1 = eval(grid1_meta["LowerRightMtrs"])

        # Interpolate points between corners, inclusive of bounds
        shape = AOD_47.info()[2]
        x = np.linspace(x0, x1, shape[2], endpoint=True)
        y = np.linspace(y0, y1, shape[1], endpoint=True)

        # Return two 2D arrays representing X & Y coordinates of all points
        xv, yv = np.meshgrid(x, y)


        ###############################################################

        """ Reprojecting data into ESPG:4326 format """

        crs_info = {
            "crs": grid1_meta["Projection"],
            "crs_params": eval(grid1_meta["ProjParams"])
        }

        projfrom = "+proj={} +lon_0=0 +x_0=0 +y_0=0 +a={} +units=m +no_defs".format(
            'sinu',crs_info['crs_params'][0])
        projto = CRS.from_epsg("4326")

        transformer = Transformer.from_crs(
                projfrom,
                projto,
                always_xy=True,
            )
        lon, lat = transformer.transform(xv, yv)

        ###################################################################

        """ Data calibration and collection"""

        def data_calib(dataset):

            shape = dataset.info()[2]
            calib_info = dataset.attributes()
            data = dataset.get()

            invalid = (
                    (data < calib_info["valid_range"][0]) |
                    (data > calib_info["valid_range"][1]) |
                    (data == calib_info["_FillValue"])
                )

            masked_data = np.ma.masked_where(invalid, data)
            

            try:
                masked_data = (
                    (masked_data - calib_info["add_offset"]) *
                    calib_info["scale_factor"]
                )
            except:
                pass

            if not calib_info['long_name'] == 'AOD_QA':
                masked_data.fill_value = np.nan
                masked_data = np.mean(masked_data, axis= 0)
                #masked_data = np.max(masked_data, axis= 0)
                
                
            return masked_data

        ###################################################################

        """ Searching for data near grid closest to given location """

        lat_min = lat.min()
        lat_max = lat.max()
        lon_min = lon.min()
        lon_max = lon.max()


        def search_loc(inp_lat, inp_lon):
            R=6371007
            lat1=np.radians(inp_lat)
            lat2=np.radians(lat)
            delta_lat=np.radians(lat-inp_lat)
            delta_lon=np.radians(lon-inp_lon)
            a=(np.sin(delta_lat/2))*(np.sin(delta_lat/2))+(np.cos(lat1))*(np.cos(lat2))*(np.sin(delta_lon/2))*(np.sin(delta_lon/2))
            c=2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
            d=R*c
            x,y=np.unravel_index(d.argmin(),d.shape)
            return lat[x,y], lon[x,y], x,y


        def grid_mean(calib_data,x,y,n):

            """calib data is preprocessed data
                n is the grid size:
                Example: n = 3 will given mean for grid of size 3x3
                n should be odd

            """
            num = int((n-1)/2)

            while(x < num):
                x+=1
            while(x > data.shape[0]-num - 1) :
                x+=1
            while(y < num):
                y+=1
            while(y > data.shape[1]- num - 1):
                y-=1
            grid = calib_data[x-num:x+num + 1,y-num:y+num +1]
            if grid.count() == 0:
                value = 0
                count = 0
                var = 0
            else:
                value = grid.mean()
                count = grid.count()
                var = grid.var()
            return value , count, var

        data = data_calib(AOD_47)
        cwv_data = data_calib(CWV)
        data55 = data_calib(AOD_55)
        for i in range(len(location_data)):
            inp_lat = location_data[i][0]
            inp_lon = location_data[i][1]
            a,b,x,y = search_loc(inp_lat, inp_lon)
            var = [date,
                inp_lat,
                inp_lon, 
                data[x,y],
                grid_mean(data, x, y, 3)[0], 
                grid_mean(data, x, y, 3)[1], 
                grid_mean(data, x, y, 5)[0], 
                grid_mean(data, x, y, 5)[1],
                cwv_data[x,y],
                grid_mean(data, x, y, 3)[2],
                grid_mean(data, x, y, 5)[2],
                data55[x,y] ]
            #print(data[x,y])
            ls.append(var)
        
        count+=1
        # if(count==2):
        #     break

    furnished = pd.DataFrame(ls, columns= ['date', 'latitude', 'longitude', 'AOD47', 'AOD47_3', 'valid3', 'AOD47_5', 'valid5', 'ColumnWV', 'AOD47_3var', 'AOD47_5var', 'AOD55'])
    furnished.to_csv(storage_path)

    return furnished





In [ ]:
modis_data_path = 'Dataset/Modis-2020'
filelist = os.listdir(modis_data_path)
location_data = pd.read_csv('Dataset/CPCB/loc-lan-updated.csv')
location_data = location_data.loc[:39, ['latitude', 'longitude']].astype(float).to_numpy()
storage_location = 'Dataset/AOD-csv/aod-jan-Dec-2020.csv'
doi = modis_preprocessor(filelist,location_data, storage_location, modis_data_path )

In [ ]:
modis_data_path = 'Dataset/Modis-2019'
filelist = os.listdir(modis_data_path)
location_data = pd.read_csv('Dataset/CPCB/loc-lan-updated.csv')
location_data = location_data.loc[:39, ['latitude', 'longitude']].astype(float).to_numpy()
storage_location = 'Dataset/AOD-csv/aod-jan-Dec-2019.csv'
doi = modis_preprocessor(filelist,location_data, storage_location, modis_data_path )